<a href="https://colab.research.google.com/github/peculab/PythonAI4Beginners/blob/main/%E7%AC%AC%E5%8D%81%E4%B8%89%E9%80%B1_%E6%96%87%E5%AD%97%E5%88%86%E9%A1%9E_Part1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**大家為什麼會來上這門課**
https://github.com/peculab/PythonAI4Beginners/issues/1

In [10]:
from google.colab import auth
auth.authenticate_user()

import gspread
from google.auth import default
creds, _ = default()

gc = gspread.authorize(creds)

In [11]:
import pandas as pd
# read data and put it in a dataframe
# 在 google 工作表載入 gsheets
gsheets = gc.open_by_url('https://docs.google.com/spreadsheets/d/1kvaMMSudWLyBV3VWOe09bee9-GQFUxSQWayuVYvVTds/edit?usp=sharing').sheet1

In [12]:
# 讀取所有數據
rows = gsheets.get_all_records()
import pandas as pd
df = pd.DataFrame(rows)

In [13]:
df.head()

,Account,Date,Cleaned_Content
0,peculab,"Aug 27, 2024",Owner peculab • 學校、姓名、以及你為什麼來上這門課？並留下你的第一個 co...
1,pupupeter,"Sep 2, 2024",Collaborator pupupeter • 高中生們兒早哇，我是這次的助教，我叫家瑋...
2,Junxian0829,"Sep 2, 2024",Collaborator Junxian0829 • 各位好，我是鄭鈞賢，是台師大機電系大...
3,tedlu-tw,"Sep 2, 2024",Collaborator tedlu-tw 各位好，和大家一樣是高中生，就讀北市數位實中，...
4,Eric970308-123,"Sep 2, 2024",Eric970308-123 大家好，我是胡程勛，目前高二，就讀彰化縣溪湖高中207班，在...


In [76]:
df.describe()

,Date
count,312
mean,2024-09-04 14:36:55.384615424
min,2024-08-27 00:00:00
25%,2024-09-03 00:00:00
50%,2024-09-05 00:00:00
75%,2024-09-05 00:00:00
max,2024-09-28 00:00:00


In [15]:
!pip install jieba
!pip install snownlp

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 37.6/37.6 MB 29.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for snownlp: filename=snownlp-0.12.3-py3-none-any.whl size=37760946 sha256=ac12a06f2976ffba84e88622daf8657bff0378ef418541e7bdc71be6be7dc37d
  Stored in directory: /root/.cache/pip/wheels/43/f3/70/8990fc249efeb396007766676706f71dd3d1ca3c023ce522ce
Successfully built snownlp


In [32]:
import jieba
from snownlp import SnowNLP
import pandas as pd
from collections import Counter
import re
import plotly.express as px

In [33]:
# 只保留中文字
def extract_chinese(text):
    return ''.join(re.findall(r'[\u4e00-\u9fa5]', text))

In [57]:
df['Chinese_Content'] = df['Cleaned_Content'].apply(extract_chinese)

# 使用 Jieba 斷詞
df['Tokenized'] = df['Chinese_Content'].apply(lambda x: list(jieba.cut(x)))

# 展平成所有詞語的列表
all_words = [word for tokens in df['Tokenized'] for word in tokens if len(word) > 1]

In [58]:
word_counts = Counter(all_words)

In [59]:
# 排除停用詞（這裡提供一個簡單的停用詞列表，可根據需要擴展）
stop_words = set(['所以', '好', '因為', '大家', '的', '是', '了', '我', '也', '在', '和', '就', '不', '有', '他', '她', '你', '我們', '這個'])
filtered_word_counts = {word: count for word, count in word_counts.items() if word not in stop_words}

In [71]:
# 將詞頻轉為 DataFrame 並排序
word_freq_df = pd.DataFrame(filtered_word_counts.items(), columns=['Word', 'Frequency'])
word_freq_df = word_freq_df.sort_values(by='Frequency', ascending=False).reset_index(drop=True)

# 繪製關鍵字統計的條形圖
fig = px.bar(word_freq_df.head(30), x='Word', y='Frequency',
             title='關鍵字出現次數統計（前20名）',
             labels={'Word': '關鍵字', 'Frequency': '出現次數'},
             text='Frequency')

fig.update_layout(
    xaxis_title="關鍵字",
    yaxis_title="出現次數",
    xaxis=dict(showgrid=True),
    yaxis=dict(showgrid=True),
    template='plotly_white'
)

fig.show()

In [67]:
!apt-get update -qq
!apt-get install -y fonts-wqy-zenhei

W: Skipping acquire of configured file 'main/source/Sources' as repository 'https://r2u.stat.illinois.edu/ubuntu jammy InRelease' does not seem to provide it (sources.list entry misspelt?)
Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following NEW packages will be installed:
  fonts-wqy-zenhei
0 upgraded, 1 newly installed, 0 to remove and 56 not upgraded.
Need to get 7,472 kB of archives.
After this operation, 16.8 MB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu jammy/universe amd64 fonts-wqy-zenhei all 0.9.45-8 [7,472 kB]
Fetched 7,472 kB in 2s (4,249 kB/s)
Selecting previously unselected package fonts-wqy-zenhei.
(Reading database ... 123629 files and directories currently installed.)
Preparing to unpack .../fonts-wqy-zenhei_0.9.45-8_all.deb ...
Unpacking fonts-wqy-zenhei (0.9.45-8) ...
Setting up fonts-wqy-zenhei (0.9.45-8) ...
Processing triggers for fontconfig (2.13.1-4.2ubuntu5) ...


In [70]:
from wordcloud import WordCloud

# 取前 300 個最常出現的詞
top_words = word_freq_df.head(300)

# 使用 WQY Zenhei 字體
wordcloud = WordCloud(
    width=800,
    height=400,
    background_color='white',
    font_path='/usr/share/fonts/truetype/wqy/wqy-zenhei.ttc'
)

# 根據詞頻生成文字雲
wordcloud.generate_from_frequencies(dict(zip(top_words['Word'], top_words['Frequency'])))

# 將文字雲轉換為圖像
wordcloud_img = wordcloud.to_array()

# 使用 Plotly 顯示文字雲
fig = px.imshow(wordcloud_img)
fig.update_layout(
    title="文字雲",
    xaxis=dict(showgrid=False, showticklabels=False, zeroline=False),
    yaxis=dict(showgrid=False, showticklabels=False, zeroline=False),
    coloraxis_showscale=False
)
fig.show()

In [63]:
# 按日期計算討論熱度
df['Date'] = pd.to_datetime(df['Date'])
heatmap_data = df.groupby('Date').size().reset_index(name='Discussion_Count')

# 繪製趨勢圖
fig = px.line(heatmap_data, x='Date', y='Discussion_Count',
              title='不同時間段的討論熱度趨勢',
              labels={'Date': '日期', 'Discussion_Count': '討論數量'},
              markers=True)

fig.update_layout(
    xaxis_title="日期",
    yaxis_title="討論數量",
    xaxis=dict(showgrid=True),
    yaxis=dict(showgrid=True),
    template='plotly_white'
)

fig.show()

In [73]:
from google.colab import userdata
apikey = userdata.get('GOOGLE_API_KEY')

In [75]:
import google.generativeai as genai
genai.configure(api_key = apikey)
model = genai.GenerativeModel("gemini-1.5-flash")

prompt = f"請摘要出這些留言的重點：{df.to_string()}"

response = model.generate_content(prompt)
print(response.text)

這些留言是學生和老師在一個Python和AI課程的留言，主要內容包含：

* **自我介紹:**  大多數留言是學生自我介紹，包括姓名、學校、年級、班級，以及他們選修這門課的原因。  許多學生來自不同的高中，例如中山附中、溪湖高中、南投高中、鼓山高中、仁武高中等等，也有少數來自高職和大學。

* **選課動機:**  學生選課的動機大多圍繞著對程式設計、AI、資訊科技的興趣，以及希望將這些技能應用於未來升學、就業或生活。部分學生是因為課程流行趨勢、老師推薦、朋友邀請或學校安排而選課。  也有一些學生希望藉此提升學習歷程檔案或利用程式解決實際問題(例如：寫作業，設計遊戲，分析資料)。

* **程式碼分享:**  許多學生分享了他們自己編寫或從助教那裡複製的程式碼連結，這些程式碼大多是在Google Colab上運行。

* **助教互動:**  助教（pupupeter）在留言中提醒學生開啟程式碼的共用權限，並鼓勵學生互相交流。

* **老師互動:**  一位數學老師(yehtzuchun)留言表示對學生用Python解決數學問題感到讚賞。

* **其他:**  少數留言包含一些額外的資訊，例如一個非學生的參與者說明其選課動機，以及一些學生的個人興趣愛好。


總體而言，留言展現了學生對程式設計和AI的熱情以及對課程的期待。  許多學生是程式設計的初學者，希望透過這門課學習基礎知識並提升技能。

